In [113]:
# Before running these commands, create a virtual environment (outside project directory) using python -m venv "dir_name/env_name"
%pip install deepxde
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [114]:
%pip install chart-studio
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [115]:
import deepxde as dde
import numpy as np

In [ ]:
geo = dde.geometry.Interval(-1, 1)
T = 0.5
def dU(x):
    return 4*(x[:, 0]**2 - 1)*(x[:, 0])

def committor(x, q):
    dq_x = dde.gradients.jacobian(q, x)
    dlap_q = dde.gradients.hessian(q, x)
    grad_U = dU(x)
    beta = 1 / T
    return -1/beta * dlap_q + grad_U * dq_x


bc_a = dde.icbc.DirichletBC(geo, lambda x: 0.0, lambda x, on_boundary: on_boundary and np.isclose(x[0], -1))
bc_b = dde.icbc.DirichletBC(geo, lambda x: 1.0, lambda x, on_boundary: on_boundary and np.isclose(x[0], 1))


In [111]:
data = dde.data.pde.TimePDE(
    geo,
    committor, 
    [bc_a, bc_b], 
    num_domain=1000,
    num_boundary=2,
    num_test=500
)

In [112]:
net = dde.maps.FNN([1] + [64] * 3 + [1], "tanh", "Glorot normal")
model = dde.Model(data, net)
model.compile("adam", lr=0.001)
losshistory, train_state = model.train(iterations=2000)

Compiling model...
Building feed-forward neural network...
'build' took 0.412130 s

'compile' took 2.154755 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [4.74e-02, 3.89e-02, 6.44e-01]    [4.75e-02, 3.89e-02, 6.44e-01]    []  
1000      [1.18e-01, 3.57e-02, 3.57e-02]    [1.18e-01, 3.57e-02, 3.57e-02]    []  
2000      [1.28e-01, 3.11e-02, 3.08e-02]    [1.28e-01, 3.11e-02, 3.08e-02]    []  

Best model at step 1000:
  train loss: 1.89e-01
  test loss: 1.89e-01
  test metric: []

'train' took 53.025429 s



In [104]:
model.compile("L-BFGS-B")
losshistory, train_state=model.train()

Compiling model...
(None, None)
(None, None)
'compile' took 1.155977 s

Training model...

Step      Train loss                        Test loss                         Test metric
2000      [1.28e-01, 3.11e-02, 3.08e-02]    [1.28e-01, 3.11e-02, 3.08e-02]    []  
INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  Objective function value: 0.189543
  Number of iterations: 1
  Number of functions evaluations: 31
2019      [1.28e-01, 3.11e-02, 3.08e-02]    [1.28e-01, 3.11e-02, 3.08e-02]    []  

Best model at step 1000:
  train loss: 1.89e-01
  test loss: 1.89e-01
  test metric: []

'train' took 5.190391 s



In [109]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

x = np.linspace(-1, 1, 100).reshape(-1, 1)
x_tf = tf.convert_to_tensor(x, dtype=tf.float32)
q_tf = tf.convert_to_tensor(model.predict(x), dtype=tf.float32)
res = committor(x_tf, q_tf)
u_pred = model.predict(x)

plt.plot(x, u_pred, label="q(x)")
plt.xlabel("x")
plt.ylabel("Committor q(x)")
plt.title("Committor function: P[x hits 1 before -1]")
plt.grid(True)
plt.legend()
plt.show()
# OK plan stay up and look into this like there is no tomorrow

AttributeError: 'NoneType' object has no attribute 'shape'